In [1]:
import sys
sys.path.append("..")

from pathlib import Path

from mndot_bid_etl.reader.abstract import create_abstract_data_from_csv
from mndot_bid_etl.reader.item import create_item_data_from_csv
from mndot_bid_etl.transform.functional.transform_bid import transform_bid_df
from mndot_bid_etl.transform.functional.transform_bidder import transform_bidder_df
from mndot_bid_etl.transform.functional.transform_contract import transform_contract_df

In [2]:
abstract_data = create_abstract_data_from_csv(Path("../data/csv/2022/220002.csv").resolve())

item_data_2020 = create_item_data_from_csv(Path("../data/csv/item_list_2020.csv").resolve())

# New transformation logic to match API

In [3]:
df =abstract_data.bid_df
df.head()

,ContractId,SectionDescription,LineNumber,ItemNumber,ItemDescription,UnitPrice,Quantity,UnitName,Engineers (Unit Price),Engineers (Extended Amount),0000198793 (Unit Price),0000198793 (Extension),0000210000 (Unit Price),0000210000 (Extension),0000207897 (Unit Price),0000207897 (Extension)
0,220002,0001 - SP 0803-44,0005,2011601/01000,AS BUILT,15000,1,LS,15000,15000,"$9,800.00","$9,800.00","$12,000.00","$12,000.00","$9,800.00","$9,800.00"
1,220002,0001 - SP 0803-44,0010,2021501/00010,MOBILIZATION,250000,1,LS,250000,250000,"$209,000.00","$209,000.00","$317,000.00","$317,000.00","$216,000.00","$216,000.00"
2,220002,0001 - SP 0803-44,0015,2051501/00010,MAINT AND RESTORATION OF HAUL ROADS,1000,1,LS,1000,1000,$1.00,$1.00,$1.00,$1.00,$1.00,$1.00
3,220002,0001 - SP 0803-44,0020,2101505/00020,CLEARING,8000,1.4,ACRE,8000,11200,"$3,500.00","$4,900.00","$3,500.00","$4,900.00","$8,000.00","$11,200.00"
4,220002,0001 - SP 0803-44,0025,2101505/00030,GRUBBING,4000,1.4,ACRE,4000,5600,"$3,700.00","$5,180.00","$3,700.00","$5,180.00","$5,500.00","$7,700.00"


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   ContractId                   142 non-null    string
 1   SectionDescription           142 non-null    string
 2   LineNumber                   142 non-null    string
 3   ItemNumber                   142 non-null    string
 4   ItemDescription              142 non-null    string
 5   UnitPrice                    142 non-null    string
 6   Quantity                     142 non-null    string
 7   UnitName                     142 non-null    string
 8   Engineers (Unit Price)       142 non-null    string
 9   Engineers (Extended Amount)  142 non-null    string
 10  0000198793 (Unit Price)      142 non-null    string
 11  0000198793 (Extension)       142 non-null    string
 12  0000210000 (Unit Price)      142 non-null    string
 13  0000210000 (Extension)       142 no

In [5]:
import pandas as pd
from mndot_bid_etl.dtype import DType

In [6]:
out_df = pd.DataFrame()
out_df.head()

""


In [7]:
out_df["contract_id"] = df["ContractId"].astype(DType.INT64)
out_df.head()

,contract_id
0,220002
1,220002
2,220002
3,220002
4,220002


In [8]:
out_df["spec_year"] = "2020"
out_df["spec_year"] = out_df["spec_year"].astype(DType.STRING)
out_df.head()

,contract_id,spec_year
0,220002,2020
1,220002,2020
2,220002,2020
3,220002,2020
4,220002,2020


In [9]:
out_df["spec_code"] = df["ItemNumber"].str.strip().str.slice(0, 4)
out_df["unit_code"] = df["ItemNumber"].str.strip().str.slice(4, 7)
out_df["item_code"] = df["ItemNumber"].str.strip().str.slice(8)
out_df.head()

,contract_id,spec_year,spec_code,unit_code,item_code
0,220002,2020,2011,601,01000
1,220002,2020,2021,501,00010
2,220002,2020,2051,501,00010
3,220002,2020,2101,505,00020
4,220002,2020,2101,505,00030


In [10]:
unit_price_columns = [column for column in df.columns.to_list() if "(Unit Price)" in column]

unit_price_columns

['Engineers (Unit Price)',
 '0000198793 (Unit Price) ',
 '0000210000 (Unit Price) ',
 '0000207897 (Unit Price) ']

In [11]:
bidder_id_name_map = {}

for column in unit_price_columns:
    out_column_name = column.split(" ")[0].strip()
    if out_column_name == "Engineers":
        bidder_id_name_map[column] = "0"
    else:
        bidder_id_name_map[column] = str(int(out_column_name))

bidder_id_name_map

{'Engineers (Unit Price)': '0',
 '0000198793 (Unit Price) ': '198793',
 '0000210000 (Unit Price) ': '210000',
 '0000207897 (Unit Price) ': '207897'}

In [12]:
# append unit price columns to out_df
def format_price(price: str) -> int:
    cleaned_str = price.strip().replace("$", "").replace(",", "")
    return int(float(cleaned_str) * 100)

for source_name, target_name in bidder_id_name_map.items():
    out_df[target_name] = df[source_name]
    out_df[target_name] = out_df[target_name].apply(format_price)

out_df.head()

,contract_id,spec_year,spec_code,unit_code,item_code,0,198793,210000,207897
0,220002,2020,2011,601,01000,1500000,980000,1200000,980000
1,220002,2020,2021,501,00010,25000000,20900000,31700000,21600000
2,220002,2020,2051,501,00010,100000,100,100,100
3,220002,2020,2101,505,00020,800000,350000,350000,800000
4,220002,2020,2101,505,00030,400000,370000,370000,550000


In [13]:
def format_quantity(quantity: str) -> float:
    return float(quantity.strip())

out_df["quantity"] = df["Quantity"].apply(format_quantity)
out_df.head()

,contract_id,spec_year,spec_code,unit_code,item_code,0,198793,210000,207897,quantity
0,220002,2020,2011,601,01000,1500000,980000,1200000,980000,1.0
1,220002,2020,2021,501,00010,25000000,20900000,31700000,21600000,1.0
2,220002,2020,2051,501,00010,100000,100,100,100,1.0
3,220002,2020,2101,505,00020,800000,350000,350000,800000,1.4
4,220002,2020,2101,505,00030,400000,370000,370000,550000,1.4


In [14]:
melt_df = out_df.melt(
    id_vars=["contract_id", "spec_year", "spec_code", "unit_code", "item_code", "quantity"],
    var_name="bidder_id",
    value_name="unit_price"
)
melt_df["bidder_id"] = melt_df["bidder_id"].astype(DType.INT64)

melt_df.head()

,contract_id,spec_year,spec_code,unit_code,item_code,quantity,bidder_id,unit_price
0,220002,2020,2011,601,01000,1.0,0,1500000
1,220002,2020,2021,501,00010,1.0,0,25000000
2,220002,2020,2051,501,00010,1.0,0,100000
3,220002,2020,2101,505,00020,1.4,0,800000
4,220002,2020,2101,505,00030,1.4,0,400000


In [15]:
melt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   contract_id  568 non-null    int64  
 1   spec_year    568 non-null    string 
 2   spec_code    568 non-null    string 
 3   unit_code    568 non-null    string 
 4   item_code    568 non-null    string 
 5   quantity     568 non-null    float64
 6   bidder_id    568 non-null    int64  
 7   unit_price   568 non-null    int64  
dtypes: float64(1), int64(3), string(4)
memory usage: 35.6 KB


In [16]:
winning_bidder_id = 198793

def assign_bid_type(bidder_id: int, winning_bidder_id: int) -> str:
    if bidder_id == 0:
        return "engineer"
    elif bidder_id == winning_bidder_id:
        return "winning"
    else:
        return "losing"

from functools import partial

partial_assign_bid_type = partial(assign_bid_type, winning_bidder_id=winning_bidder_id)

melt_df["bid_type"] = melt_df["bidder_id"].apply(partial_assign_bid_type).astype(DType.STRING)
melt_df.head(-50)

,contract_id,spec_year,spec_code,unit_code,item_code,quantity,bidder_id,unit_price,bid_type
0,220002,2020,2011,601,01000,1.0,0,1500000,engineer
1,220002,2020,2021,501,00010,1.0,0,25000000,engineer
2,220002,2020,2051,501,00010,1.0,0,100000,engineer
3,220002,2020,2101,505,00020,1.4,0,800000,engineer
4,220002,2020,2101,505,00030,1.4,0,400000,engineer
...,...,...,...,...,...,...,...,...,...
513,220002,2020,2531,604,00208,34.0,207897,11391,losing
514,220002,2020,2531,618,00010,593.0,207897,4725,losing
515,220002,2020,2540,602,00150,16.0,207897,24000,losing
516,220002,2020,2545,501,00101,1.0,207897,2064100,losing


In [17]:
melt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   contract_id  568 non-null    int64  
 1   spec_year    568 non-null    string 
 2   spec_code    568 non-null    string 
 3   unit_code    568 non-null    string 
 4   item_code    568 non-null    string 
 5   quantity     568 non-null    float64
 6   bidder_id    568 non-null    int64  
 7   unit_price   568 non-null    int64  
 8   bid_type     568 non-null    string 
dtypes: float64(1), int64(3), string(5)
memory usage: 40.1 KB


In [18]:
from mndot_bid_etl.transform import bid

transformed_bid_df = bid.transform_bid(df, "2020", 198793)
transformed_bid_df.head()

,contract_id,spec_year,spec_code,unit_code,item_code,quantity,bidder_id,unit_price,bid_type
0,220002,2020,2011,601,01000,1.0,0,1500000,engineer
1,220002,2020,2021,501,00010,1.0,0,25000000,engineer
2,220002,2020,2051,501,00010,1.0,0,100000,engineer
3,220002,2020,2101,505,00020,1.4,0,800000,engineer
4,220002,2020,2101,505,00030,1.4,0,400000,engineer


In [19]:
transformed_bid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568 entries, 0 to 567
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   contract_id  568 non-null    int64  
 1   spec_year    568 non-null    string 
 2   spec_code    568 non-null    string 
 3   unit_code    568 non-null    string 
 4   item_code    568 non-null    string 
 5   quantity     568 non-null    float64
 6   bidder_id    568 non-null    int64  
 7   unit_price   568 non-null    int64  
 8   bid_type     568 non-null    string 
dtypes: float64(1), int64(3), string(5)
memory usage: 40.1 KB


In [20]:
transformed_bid_df.head(-50)

,contract_id,spec_year,spec_code,unit_code,item_code,quantity,bidder_id,unit_price,bid_type
0,220002,2020,2011,601,01000,1.0,0,1500000,engineer
1,220002,2020,2021,501,00010,1.0,0,25000000,engineer
2,220002,2020,2051,501,00010,1.0,0,100000,engineer
3,220002,2020,2101,505,00020,1.4,0,800000,engineer
4,220002,2020,2101,505,00030,1.4,0,400000,engineer
...,...,...,...,...,...,...,...,...,...
513,220002,2020,2531,604,00208,34.0,207897,11391,losing
514,220002,2020,2531,618,00010,593.0,207897,4725,losing
515,220002,2020,2540,602,00150,16.0,207897,24000,losing
516,220002,2020,2545,501,00101,1.0,207897,2064100,losing
